## Step 16: 복잡한 계산 그래프 (구현 편)

앞서 Step 15에서 설명한 이론을 코드로 구현해보도록 하겠다. 해당 구현은 두 단계로 이루어진다.

- 순전파 시 세대 설정
- 역전파 시 최근 세대 함수부터 꺼내도록 설정

### 16.1 세대 추가

먼저 Variable 클래스와 Function 클래스에 인스턴스 변수 generation을 추가하겠다.

```python
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} type is not supported.')

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 세대를 기록하는 변수 generation

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대 기록: 부모 세대 +1
        ...
```

Variable 클래스에서는 generation을 0으로 초기화한다. 이후 set_creator 메서드 내에서 부모 세대보다 +1만큼 큰 세대를 설정해주었다. 이에 따라 함수에 의해 생성되는 변수는 부모 함수보다 1 큰 세대를 갖게 된다.

이제 이렇게 설정된 세대에 따라, 해당 변수를 입력 변수로 하는 함수에게 전달하는 부분을 Function 클래스에서 구현하도록 하겠다.

Function 클래스의 generation은 입력 변수와 같은 값으로 설정한다. 이때 입력 변수가 둘 이상이라면 더 큰 generation의 수를 선택한다.

```python
class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        # 입력 변수의 세대 중 더 큰 값을 함수의 세대로 설정
        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]
```

이와 같은 수정을 통해 다음과 같이 세대를 설정할 수 있게 되었다.

<img width="60%" alt="image" src="https://user-images.githubusercontent.com/76294398/178428950-4d6eb4c8-5e04-4402-9186-9ff769b0d726.png">

### 16.2 세대 순으로 꺼내기

지금까지의 수정을 반영하여 순전파 계산을 수행할 경우, 모든 함수와 변수에 세대가 설정된다. 이렇게 설정된 계산 그래프를 역전파할 때, 세대가 큰 순으로 함수를 꺼내 역전파를 수행할 수 있도록 해보겠다.

```python
generations = [2,0,1,4,2]
funcs = []

for g in generations:
    f = Function()
    f.generation = g
    funcs.append(f)

funcs.sort(key=lambda x: x.generation) # 리스트 정렬
while funcs:
    f = funcs.pop()
    print(f.generation)
```

\> 4

\> 2

\> 2

\> 1

\> 0

이와 같이 리스트의 sort 메서드를 이용하여 generation을 오름차순으로 정렬하는 방법을 사용하면 세대가 큰 순으로 pop 메서드를 이용하여 꺼내올 수 있을 것이다.

다만 우리는 세대가 가장 큰 함수를 꺼내려는 것이므로, 모든 원소를 정렬하지 않고 '우선순위 큐(priority queue)'를 사용하면 더욱 효율적으로 계산할 수 있다.

```python
from queue import PriorityQueue

class Function:
    ...
    # Function 객체 간 비교를 위한 연산 메서드 정의
    def __lt__(self, other):
        return self.generation > other.generation

generations = [2, 0, 1, 4, 2]
que = PriorityQueue()

for g in generations:
    f = Function()
    f.generation = g
    que.put(f)

while not que.empty():
    inst = que.get()
    print(inst.generation)
```

\> 4

\> 2

\> 2

\> 1

\> 0

### 16.3 Variable 클래스의 backward

Variable 클래스의 backward 메서드를 구현해보도록 하자.

```python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = PriorityQueue()
        seen_set = set() # 함수 중복 추가 방지

        def add_func(f):
            if f not in seen_set:
                funcs.put(f)
                seen_set.add(f)

        add_func(self.creator)

        while not funcs.empty():
            f = funcs.get()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_funcs(x.creator)
```

가장 큰 변화는 add_func 함수의 추가이다. 그동안은 함수를 리스트에 추가하기 위해 `funcs.append(f)`를 호출하였는데, 대신 `add_func`를 호출하게 함으로써 리스트 내 함수를 세대 순으로 정렬할 수 있도록 하였다.

여기서 약간 특이한 점은, add_func 함수가 backward 메서드 내 중첩 함수로 정의되었다는 점인데, 이는 add_func 함수가 backward 메서드 내에서만 사용되고, backward 메서드 내에서 정의된 변수인 funcs와 seen_set을 사용하기 때문이다.

### 16.4 동작 확인

이제 이번 단계에서 수정한 DeZero를 테스트 해볼 차례이다.

시험 삼아 다음과 같은 계산 그래프를 역전파해보도록 하자.

<img width="80%" alt="image" src="https://user-images.githubusercontent.com/76294398/178438512-a6384d21-74fb-4c9b-ae9b-39c0e5ccb5c1.png">

코드로는 다음과 같다.

```python
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)
```

\> 32.0

\> 64.0

이제 복잡한 계산 그래프도 성공적으로 미분할 수 있게 되었다.

다음 단계에서는 DeZero의 성능, 특히 메모리 사용량에 대해 살펴보도록 하겠다.